In [ ]:
%%capture
!pip install -U crawl4ai
!pip install nest_asyncio

In [ ]:
# Check crawl4ai version
import crawl4ai
print(crawl4ai.__version__)

0.4.247


In [ ]:
%%capture
!crawl4ai-setup

In [ ]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check...
[INIT].... → Crawl4AI 0.4.247
[TEST].... ℹ Testing crawling capabilities...
[EXPORT].. ℹ Exporting PDF and taking screenshot took 3.58s
[FETCH]... ↓ https://crawl4ai.com... | Status: True | Time: 5.35s
[SCRAPE].. ◆ Processed https://crawl4ai.com... | Time: 130ms
[COMPLETE] ● https://crawl4ai.com... | Status: True | Total: 5.48s
[COMPLETE] ● ✅ Crawling test passed!


In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import re
from typing import Dict, List

def extract_2024_links(markdown_text: str) -> Dict[str, str]:
    """
    Extract links from markdown text that contain '2024' in their title or URL.

    Args:
        markdown_text (str): The markdown text to parse

    Returns:
        Dict[str, str]: Dictionary with link titles as keys and URLs as values
    """
    # Regular expression pattern for markdown links
    # Matches [title](url) pattern and captures both title and url
    link_pattern = r'\[([^\]]+)\]\(([^\)]+)\)'

    # Find all links in the text
    links = re.findall(link_pattern, markdown_text)

    # Filter links that contain '2024' and create dictionary, excluding Portal links
    links_2024 = {}
    portal_pattern = "Portal%3ACurrent_events"
    portal_event_pattern = "Portal:Current_events"
    wiki_category = "wiki/Category"
    wiki_news = "wiki/Wikinews:2024"
    wiki_portal = "wiki/Portal_talk"

    for title, url in links:
        if '2024' in url:
            # Extract the wiki page name from the complex URL structure
            wiki_page = url.split('/wiki/')[-1].rstrip('>')

            # Remove quoted title at the end if present
            wiki_page = wiki_page.split('> "')[0]

            # Fix escaped parentheses format
            wiki_page = wiki_page.replace('\\(', '(').replace('\\', ')')

            # Pattern 2: \(2024\
            wiki_page = wiki_page.replace('\\(', '(').replace('\\', ')')

            # Create the clean Wikipedia article URL
            clean_url = f"https://en.wikipedia.org/wiki/{wiki_page}"
            if portal_pattern not in clean_url and portal_event_pattern not in clean_url and wiki_category not in clean_url and wiki_news not in clean_url and wiki_portal not in clean_url :
                links_2024[title] = clean_url

    return links_2024

def print_links_formatted(links: Dict[str, str]) -> None:
    """
    Print the links in a formatted way.

    Args:
        links (Dict[str, str]): Dictionary of links to print
    """
    print("\nFound 2024 Links:")
    print("-" * 80)
    for title, url in links.items():
        print(f"Title: {title}")
        print(f"URL: {url}")
        print("-" * 80)

In [ ]:
import asyncio
from crawl4ai import *
import os

async def main():
    # Create an async context manager for the web crawler
    async with AsyncWebCrawler() as crawler:
        # Fetch the webpage content
        result = await crawler.arun(
            url="https://en.wikipedia.org/wiki/Portal%3ACurrent_events/December_2024?utm_source=chatgpt.com",
        )

        # Extract 2024 links from the markdown
        links_2024 = extract_2024_links(result.markdown)

        # Print the results
        print_links_formatted(links_2024)

        # Define the output filename
        output_file = "December.txt"

        try:
            # Open the file in write mode ('w')
            # Using 'with' ensures the file is properly closed after writing
            with open(output_file, "w", encoding="utf-8") as file:
                # Write the markdown content to the file
                file.write(result.markdown)
            print(f"\nSuccessfully saved markdown content to {output_file}")
            return links_2024

        except IOError as e:
            # Handle potential file writing errors
            print(f"Error saving file: {e}")

links_2024 = asyncio.run(main())

[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://en.wikipedia.org/wiki/Portal%3ACurrent_eve... | Status: True | Time: 0.05s
[COMPLETE] ● https://en.wikipedia.org/wiki/Portal%3ACurrent_eve... | Status: True | Total: 0.12s

Found 2024 Links:
--------------------------------------------------------------------------------
Title: 2024
URL: https://en.wikipedia.org/wiki/2024
--------------------------------------------------------------------------------
Title: 2024 Syrian opposition offensives
URL: https://en.wikipedia.org/wiki/2024_Syrian_opposition_offensives
--------------------------------------------------------------------------------
Title: Battle of Aleppo
URL: https://en.wikipedia.org/wiki/Battle_of_Aleppo_(2024)
--------------------------------------------------------------------------------
Title: 2024 Aleppo University Hospital airstrike
URL: https://en.wikipedia.org/wiki/2024_Aleppo_University_Hospital_airstrike
---------------------------------------------------------------

In [ ]:
len(links_2024)

109

In [ ]:
def clean_wiki_content(content: str) -> str:
    """
    Cleans Wikipedia article content by removing specific sections and header.

    Args:
        content (str): Raw Wikipedia article content

    Returns:
        str: Cleaned article content
    """
    # First, split by "From Wikipedia, the free encyclopedia"
    if "From Wikipedia, the free encyclopedia" in content:
        content = content.split("From Wikipedia, the free encyclopedia", 1)[1].strip()

    # List of section headers that mark the end of main content
    end_sections = [
        "## See also",
        "## Notes",
        "## References",
        "## External links",
        "## Further reading",
        "## Citations",
        "## Sources",
        "## Bibliography"
    ]

    # Find the first occurrence of any end section
    end_positions = []
    for section in end_sections:
        pos = content.find(section)
        if pos != -1:
            end_positions.append(pos)

    # If any end sections were found, cut off the content there
    if end_positions:
        content = content[:min(end_positions)].strip()

    return content

In [ ]:
import asyncio
from typing import Dict, List
from crawl4ai import AsyncWebCrawler

async def validate_wiki_links(links: Dict[str, str]) -> Dict[str, str]:
    """
    Validates Wikipedia URLs and saves valid pages' content.

    Args:
        links (Dict[str, str]): Dictionary of titles and URLs

    Returns:
        Dict[str, str]: Dictionary of valid titles and URLs
    """
    valid_links = {}
    error_marker = "Wikipedia does not have an article with this exact name"

    # Create pages directory if it doesn't exist
    import os
    pages_dir = "cleaned_pages"
    os.makedirs(pages_dir, exist_ok=True)

    async with AsyncWebCrawler() as crawler:
        for title, url in links.items():
            try:
                # Create a safe filename from the title
                safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
                filename = os.path.join(pages_dir, f"{safe_title}.txt")

                # Fetch the webpage content
                result = await crawler.arun(url=url)

                # Check if the page exists
                if error_marker not in result.markdown:
                    valid_links[title] = url

                    # Save the markdown content
                    try:
                        cleaned_content = clean_wiki_content(result.markdown)
                        with open(filename, 'w', encoding='utf-8') as f:
                            f.write(cleaned_content)
                        print(f"✓ Valid: {title} - Cleaned content saved to {filename}")
                    except Exception as e:
                        print(f"✓ Valid: {title} - But failed to save content: {str(e)}")
                else:
                    print(f"✗ Invalid: {title} - Page does not exist")

            except Exception as e:
                print(f"✗ Error checking {title}: {str(e)}")

            # Add a small delay between requests
            await asyncio.sleep(1)

    return valid_links

async def process_and_validate_links(initial_links: Dict[str, str]) -> Dict[str, str]:
    """
    Process the initial links dictionary and validate each URL.

    Args:
        initial_links (Dict[str, str]): Initial dictionary of titles and URLs

    Returns:
        Dict[str, str]: Dictionary of valid titles and URLs
    """
    print("\nValidating Wikipedia URLs and saving content...")
    print("-" * 50)

    valid_links = await validate_wiki_links(initial_links)

    print("\nValidation Summary:")
    print(f"Total links: {len(initial_links)}")
    print(f"Valid links: {len(valid_links)}")
    print(f"Invalid/broken links: {len(initial_links) - len(valid_links)}")
    print(f"Content saved to: {os.path.abspath('pages')}")

    return valid_links

In [ ]:
valid_links = await process_and_validate_links(links_2024)


Validating Wikipedia URLs and saving content...
--------------------------------------------------
[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://en.wikipedia.org/wiki/2024... | Status: True | Time: 0.05s
[COMPLETE] ● https://en.wikipedia.org/wiki/2024... | Status: True | Total: 0.12s
✓ Valid: 2024 - Cleaned content saved to cleaned_pages/2024.txt
[FETCH]... ↓ https://en.wikipedia.org/wiki/2024_Syrian_oppositi... | Status: True | Time: 0.03s
[COMPLETE] ● https://en.wikipedia.org/wiki/2024_Syrian_oppositi... | Status: True | Total: 0.07s
✓ Valid: 2024 Syrian opposition offensives - Cleaned content saved to cleaned_pages/2024 Syrian opposition offensives.txt
[FETCH]... ↓ https://en.wikipedia.org/wiki/Battle_of_Aleppo_(20... | Status: True | Time: 0.02s
[COMPLETE] ● https://en.wikipedia.org/wiki/Battle_of_Aleppo_(20... | Status: True | Total: 0.08s
✓ Valid: Battle of Aleppo - Cleaned content saved to cleaned_pages/Battle of Aleppo.txt
[FETCH]... ↓ https://en.wikipedia.org/wiki/2024_A

In [ ]:
len(valid_links)

109

In [ ]:
valid_links

{'2024': 'https://en.wikipedia.org/wiki/2024',
 '2024 Syrian opposition offensives': 'https://en.wikipedia.org/wiki/2024_Syrian_opposition_offensives',
 'Battle of Aleppo': 'https://en.wikipedia.org/wiki/Battle_of_Aleppo_(2024)',
 '2024 Aleppo University Hospital airstrike': 'https://en.wikipedia.org/wiki/2024_Aleppo_University_Hospital_airstrike',
 '2024 North Indian Ocean cyclone season': 'https://en.wikipedia.org/wiki/2024_North_Indian_Ocean_cyclone_season',
 '2024 Nzérékoré stampede': 'https://en.wikipedia.org/wiki/2024_Nz%C3%A9r%C3%A9kor%C3%A9_stampede',
 '2024 Icelandic parliamentary election': 'https://en.wikipedia.org/wiki/2024_Icelandic_parliamentary_election',
 '2024 Liechtenstein referendums': 'https://en.wikipedia.org/wiki/2024_Liechtenstein_referendums#1_December',
 '2024 Romanian parliamentary election': 'https://en.wikipedia.org/wiki/2024_Romanian_parliamentary_election',
 'ceasefire': 'https://en.wikipedia.org/wiki/2024_Israel%E2%80%93Lebanon_ceasefire_agreement',
 '202

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install langchain openai tqdm langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
import json
from tqdm import tqdm

def read_file_with_name(file_path):
    """Read file content and extract title from filename."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Extract title from filename (remove .txt and replace underscores/hyphens with spaces)
    title = os.path.basename(file_path).replace('.txt', '').replace('_', ' ').replace('-', ' ')

    return content, title

def create_chunks(text, chunk_size=1000, chunk_overlap=200):
    """Create chunks from text with specified size and overlap."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False
    )
    return text_splitter.split_text(text)

def get_dynamic_prompt(title: str) -> PromptTemplate:
    """Generate a dynamic prompt template based on the article title."""
    prompt_template = f"""
    You are processing a Wikipedia article about {title}. Your primary goal is to preserve valuable information while improving readability and discarding non-informative content.

    Please analyze the following text chunk and determine if it contains meaningful information. Then provide:

    1. An assessment of the content quality:
       - Does it contain actual information about {title}?
       - Is it more than just links, references, or navigation elements?
       - Does it provide meaningful context or details?

    2. If the content is meaningful, provide:
       - A clear, well-written version that MUST:
         * Preserve all important facts, details, and specific information
         * Maintain all dates, numbers, names, and technical details
         * Remove Wikipedia-specific formatting and reference links
         * Use appropriate tone for the subject matter
       - A relevant instruction or question that tests understanding

    Critical Guidelines:
    - If the chunk contains mostly links, navigation, or non-informative content, mark it for discarding
    - For meaningful content, preserve ALL important information
    - Remove only Wikipedia elements like [1], [2] or reference links
    - Maintain technical accuracy and detail level
    - Keep important quotes and attributions

    Format your response as a JSON object with these exact keys:
    - "is_meaningful": boolean indicating if the content contains valuable information
    - "enhanced_text": The improved text if meaningful, null if not
    - "question": A specific question that:
        * Uses clear context from the content
        * Includes the title only if it adds clarity (e.g., "Jeju Air Flight 2216" for flight-specific details)
        * Avoids the title if the context is clear without it
        * References specific details, numbers, or events from the text
    - "reasoning": Brief explanation of why the content was kept or discarded

    Example questions:
    Good: "How many passengers were on board Jeju Air Flight 2216 when the incident occurred?"
    Good: "What was the initial police assessment of casualties in the New Orleans attack?"
    Bad: "What happened next?" (too vague, no context)
    Bad: "Describe the events of the 2024 incident." (too general)

    Text to process: {{text}}

    Respond only with the JSON object.
    """

    return PromptTemplate(
        input_variables=["text"],
        template=prompt_template
    )

def parse_llm_response(response_text):
    """Parse LLM response and handle various JSON formats."""
    try:
        # Clean up the response text
        response_text = response_text.strip()
        if response_text.startswith("```json"):
            response_text = response_text[7:]
        if response_text.endswith("```"):
            response_text = response_text[:-3]

        # Parse JSON
        parsed = json.loads(response_text.strip())

        # Validate required fields
        required_fields = ["is_meaningful", "enhanced_text", "question", "reasoning"]
        if not all(field in parsed for field in required_fields):
            raise ValueError("Missing required fields in JSON response")

        # Validate question format
        if parsed["is_meaningful"] and parsed["question"]:
            if len(parsed["question"]) < 20:  # Basic length check
                raise ValueError("Question too short or lacks context")
            if parsed["question"].startswith("What is") and len(parsed["question"]) < 30:
                raise ValueError("Question too generic")

        return parsed
    except Exception as e:
        print(f"Error parsing LLM response: {e}")
        return None

def process_chunks(chunks, title, llm):
    """Process text chunks through LLM with quality filtering."""
    dataset_dict = {}
    prompt = get_dynamic_prompt(title)

    for idx, chunk in enumerate(tqdm(chunks)):
        # Create full prompt
        full_prompt = prompt.format(text=chunk)

        # Get LLM response
        try:
            llm_response = llm(full_prompt)

            parsed_response = parse_llm_response(llm_response.content)
            print(parsed_response)

            if parsed_response and parsed_response["is_meaningful"]:
                dataset_dict[f"chunk_{idx}"] = {
                    "original_text": chunk,
                    "enhanced_text": parsed_response["enhanced_text"],
                    "question": parsed_response["question"],  # Updated from instruction to question
                    "title": title,
                    "reasoning": parsed_response["reasoning"]
                }
                print(f"\n✓ Chunk {idx} processed and kept")
                print(f"Question: {parsed_response['question']}")  # Display the generated question
            else:
                print(f"\n✗ Chunk {idx} discarded - not meaningful")
                print(chunk)

        except Exception as e:
            print(f"\n✗ Error processing chunk {idx}: {e}")
            continue

    return dataset_dict

def save_dataset(dataset_dict, output_file):
    """Save the processed dataset to a JSON file."""
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dataset_dict, f, ensure_ascii=False, indent=4)

def main(pages_dir="cleaned_pages"):
    """Main function to process all cleaned Wikipedia pages."""
    all_datasets = {}

    # Process each file in the cleaned pages directory
    for filename in os.listdir(pages_dir):
        if filename.endswith(".txt"):
            print("============")
            print(filename)
            print("============")
            file_path = os.path.join(pages_dir, filename)
            print(f"\nProcessing {filename}...")

            # Read and process the file
            text, title = read_file_with_name(file_path)
            chunks = create_chunks(text)

            # Setup LLM
            llm = ChatOpenAI(temperature=0.3, model_name="gpt-4o")

            # Process chunks
            file_dataset = process_chunks(chunks, title, llm)

            # Add to main dataset
            all_datasets.update(file_dataset)
        break

    # Save the complete dataset
    save_dataset(all_datasets, "enhanced_wiki_dataset.json")
    print(f"\nDataset created with {len(all_datasets)} entries")

    # Create Hugging Face dataset
    dataset_dict_hf = {
        "original_text": [],
        "enhanced_text": [],
        "question": [],  # Updated from instruction to question
        "title": [],
        "reasoning": []
    }

    for chunk_data in all_datasets.values():
        for key in dataset_dict_hf.keys():
            dataset_dict_hf[key].append(chunk_data.get(key, None))

    from datasets import Dataset
    dataset = Dataset.from_dict(dataset_dict_hf)

    return dataset

In [ ]:
dataset = main()

2024 Cape Mendocino earthquake.txt

Processing 2024 Cape Mendocino earthquake.txt...


  0%|          | 0/265 [00:00<?, ?it/s]

{'is_meaningful': False, 'enhanced_text': None, 'question': None, 'reasoning': 'The provided text is primarily a navigation element indicating a redirect from a Wikipedia page. It does not contain any actual information about the 2024 Cape Mendocino earthquake, such as details, context, or facts. Therefore, it is marked for discarding as it lacks meaningful content.'}

✗ Chunk 0 discarded - not meaningful
(Redirected from [2024 Cape Mendocino earthquake](https://en.wikipedia.org/wiki/</w/index.php?title=2024_Cape_Mendocino_earthquake&redirect=no> "2024 Cape Mendocino earthquake"))


  0%|          | 1/265 [00:01<08:45,  1.99s/it]

{'is_meaningful': False, 'enhanced_text': None, 'question': None, 'reasoning': 'The text provided consists primarily of links, images, and navigation elements related to a list of earthquakes in 2024. It does not contain any specific information or details about the 2024 Cape Mendocino earthquake or any other earthquake events. Therefore, it lacks meaningful content and is marked for discarding.'}

✗ Chunk 1 discarded - not meaningful
Earthquakes in 2024[![List of earthquakes in 2024 is located in Earth](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/World_location_map_%28equirectangular_180%29.svg/300px-World_location_map_%28equirectangular_180%29.svg.png)](https://en.wikipedia.org/wiki/</wiki/File:World_location_map_\(equirectangular_180\).svg> "List of earthquakes in 2024 is located in Earth")![List of earthquakes in 2024](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Green_pog.svg/8px-Green_pog.svg.png)![List of earthquakes in 2024](https://upload.wikimedia.o

...

{'is_meaningful': False, 'enhanced_text': None, 'question': None, 'reasoning': 'The text primarily consists of links and reference markers without any substantive information about the 2024 Cape Mendocino earthquake. It lacks context, details, or any meaningful content that could be preserved or enhanced for readability.'}

✗ Chunk 263 discarded - not meaningful
and ground cracks in the [Awash Fentale](https://en.wikipedia.org/wiki/</wiki/Awash_Fentale_\(woreda\)> "Awash Fentale \(woreda\)")-[Dulecha](https://en.wikipedia.org/wiki/</wiki/Dulecha> "Dulecha") area.[[422]](https://en.wikipedia.org/wiki/<#cite_note-422>)[[423]](https://en.wikipedia.org/wiki/<#cite_note-423>)[[424]](https://en.wikipedia.org/wiki/<#cite_note-424>)[[425]](https://en.wikipedia.org/wiki/<#cite_note-425>) | -  | -


100%|█████████▉| 264/265 [10:54<00:02,  2.09s/it]

{'is_meaningful': True, 'enhanced_text': 'An earthquake with a magnitude of 5.5 occurred 11 km (6.8 mi) southeast of Bangui in the Ilocos Region of the Philippines. The earthquake caused several buildings to be damaged, some severely, in the Bangui, Pasuquin, and Sarrat areas.', 'question': 'What was the magnitude of the earthquake that affected the Bangui, Pasuquin, and Sarrat areas in the Philippines?', 'reasoning': 'The content provides specific and valuable information about an earthquake event, including its magnitude, location, and the extent of damage. It is not merely a collection of links or navigation elements, making it meaningful for preservation and further analysis.'}

✓ Chunk 264 processed and kept
Question: What was the magnitude of the earthquake that affected the Bangui, Pasuquin, and Sarrat areas in the Philippines?


100%|██████████| 265/265 [10:57<00:00,  2.48s/it]



Dataset created with 118 entries


In [ ]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)


In [ ]:
dataset.push_to_hub("akashmaggon/calope_earthquake")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/akashmaggon/calope_earthquake/commit/9460758f5f87f80aeafab744801d39c71293ccd6', commit_message='Upload dataset', commit_description='', oid='9460758f5f87f80aeafab744801d39c71293ccd6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/akashmaggon/calope_earthquake', endpoint='https://huggingface.co', repo_type='dataset', repo_id='akashmaggon/calope_earthquake'), pr_revision=None, pr_num=None)